# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1494824033.log


This non-commercial license of GraphLab Create for academic use is assigned to cbrodeur@gmail.com and will expire on March 14, 2018.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.701349     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.744568     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print np.poly1d(w)

In [7]:
print_coefficients(model_all)
print '\n'
print model_all.coefficients.print_rows(num_rows=18)

Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
def train_model(l1_penalty, output_name, features_list):

    # Train the linear regression model on the training set
    model = graphlab.linear_regression.create(training, 
                                              target = output_name, 
                                              features = all_features, 
                                              l2_penalty = 0.,
                                              l1_penalty = l1_penalty,
                                              validation_set = None, 
                                              verbose = False)

    # Calculate the residuals and the RSS for the validation set
    residuals = validation[output_name] - model.predict(validation)
    RSS = (residuals**2).sum()   
    
    return RSS, model['coefficients']['value'].nnz()

In [10]:
l1_penalty_list = np.logspace(1, 7, num=13)

best_l1_penalty_index = -1
lowest_RSS = -1
RSS_list = []
num_nonzero_list = []

print "Calculating..."

for i in range(len(l1_penalty_list)) :
    print("i : ") + str(i)

    current_RSS, num_nonzero = train_model(l1_penalty_list[i], 'price', all_features)

    if (i == 0) :    # This is our first time through the loop, so initialize the best_l1_penalty_index and lowest_RSS
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)
        
    elif (current_RSS < lowest_RSS) :
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)        

    RSS_list.append(current_RSS)
    num_nonzero_list.append(num_nonzero)

print("\nThe optimal l1_penalty is ") + str(l1_penalty_list[best_l1_penalty_index]) + " with a RSS of " + str(lowest_RSS) 

results = graphlab.SFrame()
results['l1_penalty'] = l1_penalty_list
results['num_nonzero'] = num_nonzero_list
results['RSS'] = RSS_list
results.print_rows(num_rows=20)

Calculating...
i : 0
- New lowest RSS             : 6.25766285142e+14
  New lowest l1_penaly index : 0
i : 1
i : 2
i : 3
i : 4
i : 5
i : 6
i : 7
i : 8
i : 9
i : 10
i : 11
i : 12

The optimal l1_penalty is 10.0 with a RSS of 6.25766285142e+14
+---------------+-------------+-------------------+
|   l1_penalty  | num_nonzero |        RSS        |
+---------------+-------------+-------------------+
|      10.0     |      18     | 6.25766285142e+14 |
| 31.6227766017 |      18     | 6.25766285362e+14 |
|     100.0     |      18     | 6.25766286058e+14 |
| 316.227766017 |      18     | 6.25766288257e+14 |
|     1000.0    |      18     | 6.25766295212e+14 |
| 3162.27766017 |      18     | 6.25766317206e+14 |
|    10000.0    |      18     | 6.25766386761e+14 |
| 31622.7766017 |      18     | 6.25766606749e+14 |
|    100000.0   |      18     | 6.25767302792e+14 |
| 316227.766017 |      18     | 6.25769507644e+14 |
|   1000000.0   |      18     | 6.25776517727e+14 |
| 3162277.66017 |      18     

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

The optimal l1_penalty is 10.0 with a RSS of 6.25766285142e+14

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [11]:
optimal_model = graphlab.linear_regression.create(training, 
                                        target = 'price', 
                                        features = all_features, 
                                        l2_penalty = 0.,
                                        l1_penalty = 10.,
                                        validation_set = None, 
                                        verbose = False)

print optimal_model.coefficients.print_rows(num_rows=18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [12]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [13]:
l1_penalty_list = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [14]:
best_l1_penalty_index = -1
lowest_RSS = -1
RSS_list = []
num_nonzero_list = []

print "Calculating..."

for i in range(len(l1_penalty_list)) :
    print("i : ") + str(i)

    current_RSS, num_nonzero = train_model(l1_penalty_list[i], 'price', all_features)

    if (i == 0) :    # This is our first time through the loop, so initialize the best_l1_penalty_index and lowest_RSS
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)
        
    elif (current_RSS < lowest_RSS) :
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)        

    RSS_list.append(current_RSS)
    num_nonzero_list.append(num_nonzero)

print("\nThe optimal l1_penalty is ") + str(l1_penalty_list[best_l1_penalty_index]) + " with a RSS of " + str(lowest_RSS) 


Calculating...
i : 0
- New lowest RSS             : 6.27492659875e+14
  New lowest l1_penaly index : 0
i : 1
i : 2
i : 3
i : 4
i : 5
i : 6
i : 7
i : 8
i : 9
i : 10
i : 11
i : 12
i : 13
i : 14
i : 15
i : 16
i : 17
i : 18
i : 19

The optimal l1_penalty is 100000000.0 with a RSS of 6.27492659875e+14


In [15]:
results = graphlab.SFrame()
results['l1_penalty'] = l1_penalty_list
results['num_nonzero'] = num_nonzero_list
results['RSS'] = RSS_list
results.print_rows(num_rows=20)

+---------------+-------------+-------------------+
|   l1_penalty  | num_nonzero |        RSS        |
+---------------+-------------+-------------------+
|  100000000.0  |      18     | 6.27492659875e+14 |
|  127427498.57 |      18     | 6.28210516771e+14 |
| 162377673.919 |      18     | 6.29176689541e+14 |
| 206913808.111 |      18     | 6.30650082719e+14 |
| 263665089.873 |      17     | 6.32940229287e+14 |
| 335981828.628 |      17     |  6.3626814023e+14 |
| 428133239.872 |      17     | 6.41261198311e+14 |
| 545559478.117 |      17     | 6.48983455376e+14 |
| 695192796.178 |      17     | 6.60962217696e+14 |
|  885866790.41 |      16     | 6.77261520728e+14 |
| 1128837891.68 |      15     | 7.01046815867e+14 |
| 1438449888.29 |      15     | 7.37850622829e+14 |
| 1832980710.83 |      13     |  7.9616310964e+14 |
| 2335721469.09 |      12     | 8.69018172894e+14 |
| 2976351441.63 |      10     | 9.66925692362e+14 |
| 3792690190.73 |      6      | 1.08186759232e+15 |
| 4832930238

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 10000000000.0

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_list = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [18]:
best_l1_penalty_index = -1
lowest_RSS = -1
RSS_list = []
num_nonzero_list = []

print "Calculating..."

for i in range(len(l1_penalty_list)) :
    print("i : ") + str(i)

    current_RSS, num_nonzero = train_model(l1_penalty_list[i], 'price', all_features)

    if (i == 0) :    # This is our first time through the loop, so initialize the best_l1_penalty_index and lowest_RSS
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)
        
    elif (current_RSS < lowest_RSS) :
        lowest_RSS = current_RSS
        best_l1_penalty_index = i
        print("- New lowest RSS             : ") + str(lowest_RSS)
        print("  New lowest l1_penaly index : ") + str(best_l1_penalty_index)        

    RSS_list.append(current_RSS)
    num_nonzero_list.append(num_nonzero)

print("\nThe optimal l1_penalty is ") + str(l1_penalty_list[best_l1_penalty_index]) + " with a RSS of " + str(lowest_RSS) 

results = graphlab.SFrame()
results['l1_penalty'] = l1_penalty_list
results['num_nonzero'] = num_nonzero_list
results['RSS'] = RSS_list
results.print_rows(num_rows=20)

Calculating...
i : 0
- New lowest RSS             : 9.66925692362e+14
  New lowest l1_penaly index : 0
i : 1
i : 2
i : 3
i : 4
i : 5
i : 6
i : 7
i : 8
i : 9
i : 10
i : 11
i : 12
i : 13
i : 14
i : 15
i : 16
i : 17
i : 18
i : 19

The optimal l1_penalty is 2976351441.63 with a RSS of 9.66925692362e+14
+---------------+-------------+-------------------+
|   l1_penalty  | num_nonzero |        RSS        |
+---------------+-------------+-------------------+
| 2976351441.63 |      10     | 9.66925692362e+14 |
| 3346017155.23 |      8      | 1.03699181914e+15 |
| 3715682868.83 |      6      | 1.07436022541e+15 |
| 4085348582.43 |      6      | 1.11653797143e+15 |
| 4455014296.02 |      6      | 1.17534613768e+15 |
| 4824680009.62 |      5      |  1.2437473521e+15 |
| 5194345723.22 |      5      | 1.30055566129e+15 |
| 5564011436.82 |      3      | 1.35012188852e+15 |
| 5933677150.42 |      3      | 1.36763312119e+15 |
| 6303342864.02 |      3      | 1.40094481031e+15 |
| 6673008577.61 |      2

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [19]:
optimal_model_8 = graphlab.linear_regression.create(training, 
                                        target = 'price', 
                                        features = all_features, 
                                        l2_penalty = 0.,
                                        l1_penalty = 3346017155.23,
                                        validation_set = None, 
                                        verbose = False)

optimal_model_6 = graphlab.linear_regression.create(training, 
                                        target = 'price', 
                                        features = all_features, 
                                        l2_penalty = 0.,
                                        l1_penalty = 3715682868.83,
                                        validation_set = None, 
                                        verbose = False)

print optimal_model_8.coefficients.print_rows(num_rows=18)
print optimal_model_6.coefficients.print_rows(num_rows=18)


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 216662.306669 |  None  |
|     bedrooms     |  None | 1022.64114498 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None |  16378.182021 |  None  |
|   sqft_living    |  None | 32.8401294433 |  None  |
| sqft_living_sqrt |  None | 714.241061404 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |  604.48073943 |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 3061.95940707 |  None  |
|    sqft_above    |  None | 30.6643934582 |  None  |
|  sqft_basement   |  None |